In [1]:
import pickle
import pandas as pd
import numpy as np
import re
import ast
from flask import Flask, request, jsonify
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
with open('recipe_recommendation_model.pkl', 'rb') as f:
    vocab = pickle.load(f)

df = pd.read_csv('recipes_processed1.csv')

In [5]:
df['Ingredient_Vector'] = df['Ingredient_Vector'].apply(ast.literal_eval)

In [6]:
def vectorize_ingredients(ingredient_str, vocab):
    """ Convert input ingredients string into a vector based on vocabulary. """
    vector = [0] * len(vocab)
    ingredients = [ingredient.strip().lower() for ingredient in re.split(r',|\s', ingredient_str) if ingredient]
    for ingredient in ingredients:
        if ingredient in vocab:
            vector[vocab.index(ingredient)] = 1
    return vector

In [28]:
def recommend_recipes(user_vector, df, top_n=5):
    recipe_vectors = np.array(df['Ingredient_Vector'].tolist())
    user_vector = vectorize_ingredients(user_vector, vocab)
    user_vector = np.array(user_vector).reshape(1, -1)
    similarity_scores = cosine_similarity(user_vector, recipe_vectors)[0]
    
    recipe_similarity = list(enumerate(similarity_scores))
    top_indices = sorted(recipe_similarity, key=lambda x: x[1], reverse=True)[:top_n]
    
    results = []
    for index, score in top_indices:
        name = df.loc[index, 'Title']
        diff = df.loc[index, 'Difficulty']
        url = df.loc[index, 'Link']  
        time = df.loc[index, 'Total Cook Time'] 
        results.append({'name': name, 'Difficulty': diff, 'url': url, 'time':time})
    
    return results

In [30]:
sample_ingredients = "onion, garlic, tomato, ginger"

# Call the recommendation function
results = recommend_recipes(sample_ingredients, df, top_n=5)

# Print the results
print("Top Recipe Recommendations:\n")
for i, recipe in enumerate(results, start=1):
    print(f"{i}. {recipe['name']}")
    print(f"   Difficulty: {recipe['Difficulty']}")
    print(f"   Total time to cook: {recipe['time']}")
    print(f"   URL: {recipe['url']}\n")

Top Recipe Recommendations:

1. Black Forest Chicken Recipe
   Difficulty: easy
   Total time to cook: 30 mins
   URL: https://food.ndtv.com/recipe-black-forest-chicken-956210

2. Veg Hakka Noodles Recipe
   Difficulty: easy
   Total time to cook: 30 mins
   URL: https://food.ndtv.com/recipe-veg-hakka-noodles-952069

3. Egg Pasta Recipe
   Difficulty: easy
   Total time to cook: 25 mins
   URL: https://food.ndtv.com/recipe-egg-pasta-955600

4. Sooji Manchurian Recipe
   Difficulty: easy
   Total time to cook: 30 mins
   URL: https://food.ndtv.com/recipe-sooji-manchurian-956126

5. Mango Avocado & Ginger Bruschetta Recipe
   Difficulty: easy
   Total time to cook: 25 mins
   URL: https://food.ndtv.com/recipe-mango-avocado-ginger-bruschetta-958600

